In [45]:
import json
from pathlib import Path

import pandas as pd

In [46]:
from bellem.dvc.experiment import load_experiments

filepaths = list(Path("../../tmp/musique-sweep-semi/").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
experiments[1]

601 experiments


{'commit': 'c9ffff5dd2706e29c13cbc66cd0551cfb9dcdd1b',
 'id': '5bf0f42a3be03329be9d148689ff94f29ae6eebe',
 'name': 'irate-cons',
 'params': {'dataset': {'path': 'bdsaglam/musique-sweep',
   'name': 'answerable',
   'split': 'train'},
  'qa': {'model': 'llama-3-70b-tgi',
   'temperature': 0.1,
   'system_prompt': 'cot/original.txt',
   'user_prompt_template': 'cq.txt',
   'few_shot_examples': 'semi/cot-original.json',
   'n_shot': 0},
  'run': 2},
 'metrics': {'exact_match': 0.6033333333333334,
  'f1': 0.7033493455861877,
  'fuzzy_match': 0.6766666666666666,
  '2hops': {'exact_match': 0.71,
   'f1': 0.8188099415204678,
   'fuzzy_match': 0.81},
  '3hops': {'exact_match': 0.58,
   'f1': 0.6703333333333333,
   'fuzzy_match': 0.64},
  '4hops': {'exact_match': 0.52,
   'f1': 0.6209047619047618,
   'fuzzy_match': 0.58}}}

In [47]:
df = pd.json_normalize(experiments).drop(columns=["commit"])
mask = (
    (df["params.dataset.path"] == "bdsaglam/musique-sweep")  &
    (df["params.qa.few_shot_examples"].str.contains("semi")) &
    (df["metrics.f1"] > 0.004)
)

df = df.loc[mask].copy()
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)

df['params.qa.n_sc'] = df['params.qa.n_sc'].fillna(1)

param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]
df.dropna(subset=metric_cols, inplace=True, how="any")
df.drop_duplicates(subset=param_cols + metric_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")

372 experiments after preprocessing


In [48]:
def parse_qa_technique(row):
    base_technique = row["params.qa.system_prompt"].split("/")[0].upper()
    sc = row["params.qa.n_sc"]
    sc_suffix = f"-SC{sc}" if sc > 1 else ""
    return f"{base_technique}{sc_suffix}"

df["params.qa.technique"] = df.apply(parse_qa_technique, axis=1)

In [49]:
def sorted_tuple(x):
    return tuple(sorted(x))


run_counts = (
    df.groupby(
        [
            "params.qa.system_prompt",
            "params.qa.user_prompt_template",
            "params.qa.few_shot_examples",
            "params.qa.n_shot",
            "params.qa.n_sc",
            "params.qa.temperature",
        ]
    )["params.run"]
    .aggregate(sorted_tuple)
    .reset_index()
)
run_counts.loc[run_counts["params.run"].map(len) != 3]

,params.qa.system_prompt,params.qa.user_prompt_template,params.qa.few_shot_examples,params.qa.n_shot,params.qa.n_sc,params.qa.temperature,params.run
3,ccot/format-thought.txt,cq.txt,semi/ccot.json,0,10.0,0.5,"(1, 3)"
4,ccot/format-thought.txt,cq.txt,semi/ccot.json,0,10.0,0.7,"(1, 3)"
8,ccot/format-thought.txt,cq.txt,semi/ccot.json,1,10.0,0.1,"(1,)"
9,ccot/format-thought.txt,cq.txt,semi/ccot.json,1,10.0,0.5,"(1,)"
10,ccot/format-thought.txt,cq.txt,semi/ccot.json,1,10.0,0.7,"(1,)"
...,...,...,...,...,...,...,...
157,direct/format-few.txt,cq.txt,semi/direct.json,2,10.0,0.5,"(1,)"
158,direct/format-few.txt,cq.txt,semi/direct.json,2,10.0,0.7,"(1,)"
162,direct/format-few.txt,cq.txt,semi/direct.json,3,10.0,0.1,"(1, 3)"
163,direct/format-few.txt,cq.txt,semi/direct.json,3,10.0,0.5,"(2,)"


In [50]:
for col in run_counts.columns:
    print(f"- {col}")
    for value in run_counts[col].unique():
        print(f"\t{value}")

- params.qa.system_prompt
	ccot/format-thought.txt
	ccot/original.txt
	cok/adapted.txt
	cok/original.txt
	cot/format-thought.txt
	cot/original.txt
	cte/format-triplets-few.txt
	direct/format-few.txt
- params.qa.user_prompt_template
	cq.txt
- params.qa.few_shot_examples
	semi/ccot.json
	semi/ccot-original.json
	semi/cok.json
	semi/cot.json
	semi/cot-original.json
	semi/cte.json
	semi/direct.json
- params.qa.n_shot
	0
	1
	2
	3
- params.qa.n_sc
	1.0
	10.0
- params.qa.temperature
	0.1
	0.5
	0.7
- params.run
	(1, 2, 3)
	(1, 3)
	(1,)
	(2,)
	(1, 2)
	(2, 3)
	(3,)


In [51]:
df.to_json('exps.jsonl', orient='records', lines=True)

In [52]:
df.sort_values("metrics.f1").head(10)

,id,name,params.dataset.path,params.dataset.name,params.dataset.split,params.qa.model,params.qa.temperature,params.qa.system_prompt,params.qa.user_prompt_template,params.qa.few_shot_examples,...,metrics.exact_match,metrics.f1,metrics.2hops.exact_match,metrics.2hops.f1,metrics.3hops.exact_match,metrics.3hops.f1,metrics.4hops.exact_match,metrics.4hops.f1,params.qa.n_sc,params.qa.technique
449,2c2fcadef8502c99663e9ac454506875c325f47f,ample-esne,bdsaglam/musique-sweep,answerable,train,llama-3-70b-tgi,0.7,cok/original.txt,cq.txt,semi/cok.json,...,0.000000,0.005053,0.00,0.008205,0.00,0.006952,0.00,0.000000,1.0,COK
460,055e1c03c13fc33ed004f2121aa629831180b479,macro-tule,bdsaglam/musique-sweep,answerable,train,llama-3-70b-tgi,0.1,cok/original.txt,cq.txt,semi/cok.json,...,0.003333,0.005556,0.01,0.010000,0.00,0.000000,0.00,0.006667,1.0,COK
204,8307b632e2e91e5587426cfd957d1ab6ae43af64,tough-jean,bdsaglam/musique-sweep,answerable,train,llama-3-70b-tgi,0.5,cot/format-thought.txt,cq.txt,semi/cot.json,...,0.006667,0.006667,0.02,0.020000,0.00,0.000000,0.00,0.000000,10.0,COT-SC10.0
467,b0a52cbaec157c806bb2bff5fc04cee60e75a850,muddy-dais,bdsaglam/musique-sweep,answerable,train,llama-3-70b-tgi,0.7,cok/original.txt,cq.txt,semi/cok.json,...,0.006667,0.006667,0.01,0.010000,0.00,0.000000,0.01,0.010000,1.0,COK
206,fc14be3e2d3c0ff24bca1f364b0ce52575c17449,waxen-tosh,bdsaglam/musique-sweep,answerable,train,llama-3-70b-tgi,0.1,cot/format-thought.txt,cq.txt,semi/cot.json,...,0.006667,0.006667,0.02,0.020000,0.00,0.000000,0.00,0.000000,10.0,COT-SC10.0
447,1ff79f12577d750131b6b9c2d052534c35572be7,diazo-chin,bdsaglam/musique-sweep,answerable,train,llama-3-70b-tgi,0.1,cok/original.txt,cq.txt,semi/cok.json,...,0.006667,0.006667,0.02,0.020000,0.00,0.000000,0.00,0.000000,1.0,COK
464,b3283e1bd52840803851c3bfe0200aa037f0bb81,hyoid-tirl,bdsaglam/musique-sweep,answerable,train,llama-3-70b-tgi,0.1,cok/original.txt,cq.txt,semi/cok.json,...,0.006667,0.007920,0.01,0.010000,0.00,0.002222,0.01,0.011538,1.0,COK
488,8879aa3190b7f9e76f8ed871e1e274b62d59bc0f,dedal-kobs,bdsaglam/musique-sweep,answerable,train,llama-3-70b-tgi,0.5,cok/original.txt,cq.txt,semi/cok.json,...,0.010000,0.011113,0.02,0.022050,0.00,0.001290,0.01,0.010000,1.0,COK
48,f762a2e26510ec9fe37bd3ae1daef87a0468bbea,irony-dohs,bdsaglam/musique-sweep,answerable,train,llama-3-70b-tgi,0.7,ccot/original.txt,cq.txt,semi/ccot-original.json,...,0.010000,0.011333,0.01,0.010000,0.00,0.000000,0.02,0.024000,1.0,CCOT
482,d8d25f64766d0a3cfbf2733c7903f6a6edeb8f82,holey-cool,bdsaglam/musique-sweep,answerable,train,llama-3-70b-tgi,0.7,cok/original.txt,cq.txt,semi/cok.json,...,0.006667,0.011379,0.01,0.014000,0.01,0.016136,0.00,0.004000,1.0,COK
